# 🕐 Phase-3.1: Adaptive Time Window Manager
## Temporal Reasoning for RAG-IDS

---

### 🎯 **Objective**

Implement an adaptive temporal window manager for network flows that:
- Maintains a rolling buffer of recent flows
- Dynamically expands window when similarity density increases
- Dynamically shrinks window when patterns dissipate
- Provides O(1) amortized performance

### 📊 **Key Principles**

1. **Single flow ≠ Attack**: Decisions emerge from repeated behavior over time
2. **Adaptive sizing**: Window expands during attack sequences, shrinks during benign periods
3. **Efficiency**: O(1) amortized updates using deque data structure
4. **Explainability**: Track window state (expanding/stable/contracting)

### 🔧 **Window Behavior**

**Expansion Triggers:**
- High average similarity (>0.75)
- Repeated attack type appearances

**Shrinkage Triggers:**
- Low average similarity (<0.50)
- Stable benign patterns

**Constraints:**
- Min: 5 seconds / 20 flows
- Max: 60 seconds / 500 flows

---

## 📦 Import Required Libraries

In [2]:
from collections import deque
from dataclasses import dataclass, field
from typing import List, Dict, Any, Optional
import time
import numpy as np

print("✅ Libraries imported successfully")

✅ Libraries imported successfully


## 🏗️ Data Structures

### FlowRecord
Represents a single network flow with temporal and retrieval context.

In [3]:
@dataclass
class FlowRecord:
    """
    Represents a single network flow with its temporal and retrieval context.
    
    Attributes:
        flow_id: Unique identifier for the flow
        timestamp: Unix timestamp (seconds)
        vector_embedding: 99-dimensional feature vector
        retrieval_results: ChromaDB k-NN matches with similarity scores
        metadata: Additional flow metadata (proto, port, etc.)
    """
    flow_id: str
    timestamp: float
    vector_embedding: np.ndarray
    retrieval_results: Optional[List[Dict[str, Any]]] = None  # ChromaDB matches (can be None if not retrieved yet)
    metadata: Dict[str, Any] = field(default_factory=dict)
    
    def __post_init__(self):
        """Validate flow record after initialization."""
        if len(self.vector_embedding) != 99:
            raise ValueError(f"Expected 99-dimensional vector, got {len(self.vector_embedding)}")

print("✅ FlowRecord dataclass defined")

✅ FlowRecord dataclass defined


## 🪟 AdaptiveTimeWindow Class

Core temporal window manager with adaptive sizing.

In [4]:
class AdaptiveTimeWindow:
    """
    Adaptive temporal window manager for behavioral pattern detection.
    
    The window dynamically adjusts its size based on:
    1. Similarity density: More high-similarity matches → expand window
    2. Pattern recurrence: Repeated attack types → expand window
    3. Pattern dissipation: Low similarity or no patterns → shrink window
    """
    
    def __init__(
        self,
        min_time_seconds: float = 5.0,
        max_time_seconds: float = 60.0,
        min_flow_count: int = 20,
        max_flow_count: int = 500,
        similarity_expansion_threshold: float = 0.75,
        similarity_shrinkage_threshold: float = 0.50,
        recurrence_expansion_threshold: int = 3
    ):
        # Window constraints
        self.min_time_seconds = min_time_seconds
        self.max_time_seconds = max_time_seconds
        self.min_flow_count = min_flow_count
        self.max_flow_count = max_flow_count
        
        # Adaptation thresholds
        self.similarity_expansion_threshold = similarity_expansion_threshold
        self.similarity_shrinkage_threshold = similarity_shrinkage_threshold
        self.recurrence_expansion_threshold = recurrence_expansion_threshold
        
        # Window state
        self._buffer: deque[FlowRecord] = deque(maxlen=max_flow_count)
        self._current_window_size_target = min_time_seconds
        
        # Statistics
        self._recent_avg_similarities: deque[float] = deque(maxlen=10)
        self._attack_type_counter: Dict[str, int] = {}
        
        # Metadata
        self._total_flows_processed = 0
        self._expansion_count = 0
        self._shrinkage_count = 0
    
    def add_flow(self, flow: FlowRecord) -> None:
        """Add a new flow to the window with O(1) amortized complexity."""
        self._buffer.append(flow)
        self._total_flows_processed += 1
        self._update_statistics(flow)
        self._adjust_window_size()
        self._prune_expired_flows()
    
    def get_active_window(self) -> List[FlowRecord]:
        """Get all flows in the current active window."""
        if not self._buffer:
            return []
        
        current_time = self._buffer[-1].timestamp
        cutoff_time = current_time - self._current_window_size_target
        
        active_flows = [
            flow for flow in self._buffer
            if flow.timestamp >= cutoff_time
        ]
        
        # Enforce minimum flow count
        if len(active_flows) < self.min_flow_count and len(self._buffer) >= self.min_flow_count:
            active_flows = list(self._buffer)[-self.min_flow_count:]
        
        return active_flows
    
    def get_window_summary(self) -> Dict[str, Any]:
        """Get summary statistics for the current active window."""
        active_flows = self.get_active_window()
        
        if not active_flows:
            return {
                "flow_count": 0,
                "time_span_seconds": 0.0,
                "avg_similarity": 0.0,
                "attack_type_distribution": {},
                "window_state": "empty"
            }
        
        time_span = active_flows[-1].timestamp - active_flows[0].timestamp
        
        all_similarities = []
        attack_type_dist = {}
        
        for flow in active_flows:
            for match in flow.retrieval_results:
                similarity = match.get('similarity', match.get('distance', 0))
                all_similarities.append(similarity)
                
                attack_type = match.get('attack_type', match.get('metadata', {}).get('attack_type', 'unknown'))
                if attack_type != 'Normal':
                    attack_type_dist[attack_type] = attack_type_dist.get(attack_type, 0) + 1
        
        avg_similarity = np.mean(all_similarities) if all_similarities else 0.0
        window_state = self._get_window_state()
        
        return {
            "flow_count": len(active_flows),
            "time_span_seconds": time_span,
            "avg_similarity": float(avg_similarity),
            "attack_type_distribution": attack_type_dist,
            "window_state": window_state,
            "current_target_seconds": self._current_window_size_target,
            "total_processed": self._total_flows_processed,
            "expansion_events": self._expansion_count,
            "shrinkage_events": self._shrinkage_count
        }
    
    def reset_if_stable(self) -> bool:
        """Reset window if patterns have stabilized."""
        active_flows = self.get_active_window()
        
        if len(active_flows) < self.min_flow_count:
            return False
        
        recent_similarities = []
        for flow in active_flows[-10:]:
            for match in flow.retrieval_results:
                similarity = match.get('similarity', match.get('distance', 0))
                recent_similarities.append(similarity)
        
        avg_recent_similarity = np.mean(recent_similarities) if recent_similarities else 0.0
        
        if avg_recent_similarity < 0.3:
            self._buffer.clear()
            self._recent_avg_similarities.clear()
            self._attack_type_counter.clear()
            self._current_window_size_target = self.min_time_seconds
            return True
        
        return False
    
    def _update_statistics(self, flow: FlowRecord) -> None:
        """Update internal statistics for adaptive decision-making."""
        similarities = [
            match.get('similarity', match.get('distance', 0))
            for match in flow.retrieval_results
        ]
        avg_similarity = np.mean(similarities) if similarities else 0.0
        self._recent_avg_similarities.append(avg_similarity)
        
        for match in flow.retrieval_results:
            attack_type = match.get('attack_type', match.get('metadata', {}).get('attack_type', 'unknown'))
            if attack_type != 'Normal':
                self._attack_type_counter[attack_type] = self._attack_type_counter.get(attack_type, 0) + 1
    
    def _adjust_window_size(self) -> None:
        """Adjust window size based on recent patterns."""
        if len(self._recent_avg_similarities) < 5:
            return
        
        recent_avg = np.mean(list(self._recent_avg_similarities))
        should_expand = False
        
        if recent_avg > self.similarity_expansion_threshold:
            should_expand = True
        
        max_recurrence = max(self._attack_type_counter.values()) if self._attack_type_counter else 0
        if max_recurrence >= self.recurrence_expansion_threshold:
            should_expand = True
        
        if should_expand:
            new_target = min(
                self._current_window_size_target * 1.5,
                self.max_time_seconds
            )
            if new_target > self._current_window_size_target:
                self._current_window_size_target = new_target
                self._expansion_count += 1
        
        elif recent_avg < self.similarity_shrinkage_threshold:
            new_target = max(
                self._current_window_size_target * 0.8,
                self.min_time_seconds
            )
            if new_target < self._current_window_size_target:
                self._current_window_size_target = new_target
                self._shrinkage_count += 1
                self._attack_type_counter.clear()
    
    def _prune_expired_flows(self) -> None:
        """Remove flows older than max_time_seconds."""
        if not self._buffer:
            return
        
        current_time = self._buffer[-1].timestamp
        cutoff_time = current_time - self.max_time_seconds
        
        while self._buffer and self._buffer[0].timestamp < cutoff_time:
            old_flow = self._buffer.popleft()
            
            for match in old_flow.retrieval_results:
                attack_type = match.get('attack_type', match.get('metadata', {}).get('attack_type', 'unknown'))
                if attack_type in self._attack_type_counter:
                    self._attack_type_counter[attack_type] -= 1
                    if self._attack_type_counter[attack_type] <= 0:
                        del self._attack_type_counter[attack_type]
    
    def _get_window_state(self) -> str:
        """Determine current window state."""
        if len(self._recent_avg_similarities) < 3:
            return "initializing"
        
        recent_avg = np.mean(list(self._recent_avg_similarities)[-3:])
        
        if recent_avg > self.similarity_expansion_threshold:
            return "expanding"
        elif recent_avg < self.similarity_shrinkage_threshold:
            return "contracting"
        else:
            return "stable"
    
    def __len__(self) -> int:
        return len(self._buffer)
    
    def __repr__(self) -> str:
        summary = self.get_window_summary()
        return (
            f"AdaptiveTimeWindow("
            f"flows={summary['flow_count']}, "
            f"span={summary['time_span_seconds']:.1f}s, "
            f"state={summary['window_state']}"
            f")"
        )

print("✅ AdaptiveTimeWindow class defined")

✅ AdaptiveTimeWindow class defined


## ✅ Quick Validation Test

In [5]:
# Create window instance
window = AdaptiveTimeWindow(
    min_time_seconds=5.0,
    max_time_seconds=30.0,
    min_flow_count=10,
    max_flow_count=100
)

print("="*80)
print("Phase-3.1: Adaptive Time Window Implementation")
print("="*80)
print(f"✅ AdaptiveTimeWindow initialized")
print(f"   Min window: {window.min_time_seconds}s / {window.min_flow_count} flows")
print(f"   Max window: {window.max_time_seconds}s / {window.max_flow_count} flows")
print(f"   Initial state: {window}")
print("="*80)
print("✅ Phase-3.1 class implementation complete!")
print("   Run validation notebook for comprehensive tests.")
print("="*80)

Phase-3.1: Adaptive Time Window Implementation
✅ AdaptiveTimeWindow initialized
   Min window: 5.0s / 10 flows
   Max window: 30.0s / 100 flows
   Initial state: AdaptiveTimeWindow(flows=0, span=0.0s, state=empty)
✅ Phase-3.1 class implementation complete!
   Run validation notebook for comprehensive tests.


---

## 🎉 Phase-3.1 Complete!

### ✅ Deliverables
1. ✅ `FlowRecord` dataclass - Network flow representation
2. ✅ `AdaptiveTimeWindow` class - Core temporal window manager
3. ✅ Adaptive expansion/shrinkage logic
4. ✅ O(1) amortized performance with deque
5. ✅ Window state tracking (expanding/stable/contracting)

### 📊 Key Features
- **Dynamic sizing**: Expands (1.5×) during attacks, shrinks (0.8×) during benign periods
- **Efficient pruning**: Time-based expiration with attack counter maintenance
- **Statistics tracking**: Similarity trends and attack type recurrence
- **Reset capability**: Clears window on stable benign patterns

### 🚀 Next: Phase-3.2
ChromaDB retrieval per flow (<50ms latency requirement)

---